In [37]:
import numpy as np
import scipy.special as sp
import scipy.ndimage as sn
import matplotlib.pyplot as plt

%matplotlib inline

In [38]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes

        self.lr = learning_rate

        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        self.activation = lambda x: sp.expit(x)
        self.inverse_activation = lambda x: sp.logit(x)

        pass

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation(final_inputs)

        outputs_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, outputs_errors)

        self.who += self.lr * np.dot(outputs_errors * final_outputs * (1 - final_outputs), hidden_outputs.T)
        self.wih += self.lr * np.dot(hidden_errors * hidden_outputs * (1 - hidden_outputs), inputs.T)

        pass

    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation(final_inputs)

        return final_outputs

In [39]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1

n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [40]:
with open('../data/mnist/mnist_train.csv', 'r') as train_f:
    train_data = [line.rstrip() for line in train_f]

with open('../data/mnist/mnist_test.csv') as test_f:
    test_data = [line.rstrip() for line in test_f]

In [41]:
def test_run():
    scorecard = []

    for record in test_data:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        correct_label = int(all_values[0])
        label = np.argmax(outputs)

        if label == correct_label:
            scorecard.append(1)
        else:
            scorecard.append(0)

        pass

    scorecard_array = np.asarray(scorecard)
    print('performance:', scorecard_array.sum() / scorecard_array.size)

    pass

In [42]:
epochs = 6

for e in range(epochs):
    for record in train_data:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)

        # Image rotation on +10 degree
        inputs_plusx_img = sn.rotate(inputs.reshape(28, -1), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_plusx_img.reshape(784), targets)

        # Image rotation on -10 degree
        inputs_minusx_img = sn.rotate(inputs.reshape(28, -1), -10, cval=0.01, order=1, reshape=False)
        n.train(inputs_minusx_img.reshape(784), targets)

        pass

    test_run()
    pass

performance: 0.9633
performance: 0.9673
performance: 0.9693
performance: 0.9711
performance: 0.9716
performance: 0.9695
